# CNN

In [51]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, callbacks
from tensorflow.keras import Model, Sequential

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import string

In [52]:
#df = train
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 2 columns):
title_text    1500 non-null object
class         1500 non-null int64
dtypes: int64(1), object(1)
memory usage: 23.6+ KB


In [53]:
num_classes = df['class'].nunique()
print(num_classes)

2


In [54]:
def cleaning_text(text):
    stop_words = stopwords.words("english")

    text = re.sub(r'http\S+', " ", text)    # remove urls
    text = re.sub(r'@\w+',' ',text)         # remove mentions
    text = re.sub(r'#\w+', ' ', text)       # remove hastags
    text = re.sub('r<.*?>',' ', text)       # remove html tags
    
    # remove stopwords 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_words])

    for punctuation in string.punctuation:
        text = text.replace(punctuation, "")
    
    return text

In [55]:
df['tweet'] = df['title_text'].apply(lambda x: cleaning_text(x))

In [56]:
# Maximum sentence length
max_len_words = max(list(df['tweet'].apply(len)))
print(max_len_words)

6509


In [57]:
def tokenizer(x_train, y_train, max_len_word):
    # because the data distribution is imbalanced, "stratify" is used
    X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                      test_size=.2, shuffle=True, 
                                                      stratify=y_train, random_state=0)

    # Tokenizer
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)
    sequence_dict = tokenizer.word_index
    word_dict = dict((num, val) for (val, num) in sequence_dict.items())

    # Sequence data
    train_sequences = tokenizer.texts_to_sequences(X_train)
    train_padded = pad_sequences(train_sequences,
                                 maxlen=max_len_word,
                                 truncating='post',
                                 padding='post')
    
    val_sequences = tokenizer.texts_to_sequences(X_val)
    val_padded = pad_sequences(val_sequences,
                                maxlen=max_len_word,
                                truncating='post',
                                padding='post', )
    
    print(train_padded.shape)
    print(val_padded.shape)
    print('Total words: {}'.format(len(word_dict)))
    return train_padded, val_padded, y_train, y_val, word_dict

X_train, X_val, y_train, y_val, word_dict = tokenizer(df.tweet, df['class'], 100)

(1200, 100)
(300, 100)
Total words: 28154


In [58]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten
def build_cnn_model():
    model = Sequential()
    
    model.add(Embedding(5000, 100, input_length=100))


    model.add(Conv1D(64, 2, padding='same', activation='relu'))
    model.add(MaxPooling1D(2))
    #model.add(MaxPooling1D(2))
    
    model.add(Flatten())
    
    model.add(Dense(units=1024,activation="relu"))
    model.add(Dense(units=512,activation="relu"))
    
    model.add(Dense(units=num_classes,activation="softmax"))
    
    optimizer = Adam(lr=0.000055,beta_1=0.9,beta_2=0.999)
    
    model.compile(optimizer=optimizer,metrics=["accuracy",f1_m,precision_m, recall_m],loss=tf.keras.losses.SparseCategoricalCrossentropy())
    return model

In [59]:
cnn_model = build_cnn_model()

In [60]:
cnn_history = cnn_model.fit(X_train, y_train,
                    epochs=5, 
                    validation_data=(X_val, y_val),
                    shuffle=True)

Epoch 1/5
38/38 [==============================] - 2s 11ms/step - loss: 0.6652 - accuracy: 0.6683 - f1_m: 0.6622 - precision_m: 0.4992 - recall_m: 1.0000 - val_loss: 0.6355 - val_accuracy: 0.6367 - val_f1_m: 0.6686 - val_precision_m: 0.5052 - val_recall_m: 1.0000
Epoch 2/5
38/38 [==============================] - 0s 6ms/step - loss: 0.5557 - accuracy: 0.7500 - f1_m: 0.6645 - precision_m: 0.5016 - recall_m: 1.0000 - val_loss: 0.5322 - val_accuracy: 0.6433 - val_f1_m: 0.6686 - val_precision_m: 0.5052 - val_recall_m: 1.0000
Epoch 3/5
38/38 [==============================] - 0s 6ms/step - loss: 0.3811 - accuracy: 0.9000 - f1_m: 0.6621 - precision_m: 0.4992 - recall_m: 1.0000 - val_loss: 0.3433 - val_accuracy: 0.9200 - val_f1_m: 0.6686 - val_precision_m: 0.5052 - val_recall_m: 1.0000
Epoch 4/5
38/38 [==============================] - 0s 6ms/step - loss: 0.1606 - accuracy: 0.9942 - f1_m: 0.6594 - precision_m: 0.4975 - recall_m: 1.0000 - val_loss: 0.1589 - val_accuracy: 0.9867 - val_f1_m: 0.6

In [61]:
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [62]:
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [63]:
cnn_acc = cnn_history.history['val_accuracy'][4]
cnn_prec = cnn_history.history['val_precision_m'][4]
cnn_rec = cnn_history.history['val_recall_m'][4]
cnn_f1 = cnn_history.history['val_f1_m'][4]

In [64]:
storeResults('CNN',cnn_acc,cnn_prec,cnn_rec,cnn_f1)